## Latent Dirichlet Allocation ##
**Watch these videos:**
- https://www.youtube.com/watch?v=azXCzI57Yfc
- https://www.youtube.com/watch?v=T05t-SqKArY
- https://www.youtube.com/watch?v=BaM1uiCpj_E

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [1]:
import pandas as pd
import numpy as np

In [17]:
'''
Load the dataset from the CSV and save it to 'data_text'. We only need the'headline_text' column,
so drop the rest of the columns. Call the final dataframe "documents" 
Then, add an 'index' column to the dataframe. Be sure to name it index.
Hint: dataframe.index
'''

data_text = pd.read_csv('/content/abcnews-date-text.csv')
documents = data_text.drop('publish_date',axis=1)
documents.reset_index(drop=False,inplace=True)
documents

,index,headline_text
0,0,aba decides against community broadcasting lic...
1,1,act fire witnesses must be aware of defamation
2,2,a g calls for infrastructure protection summit
3,3,air nz staff in aust strike for pay rise
4,4,air nz strike to affect australian travellers
...,...,...
870611,870611,pauline jachmann apple cider
870612,870612,porter jet order no flight of fancy theyre wor...
870613,870613,prince charles brother in law mark shand dies
870614,870614,prince william and catherine tour canberra


Let's glance at the dataset:

In [18]:
'''
print the total number of documents
'''
len(documents)

870616

In [19]:
documents[:5]

,index,headline_text
0,0,aba decides against community broadcasting lic...
1,1,act fire witnesses must be aware of defamation
2,2,a g calls for infrastructure protection summit
3,3,air nz staff in aust strike for pay rise
4,4,air nz strike to affect australian travellers


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.

This time, we'll make use of the gensim library and show you how  to preprocess text easily. 

In [20]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [23]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [24]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [25]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [26]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    '''
        1) Lemmatize the text
        2) Then, stem the text
        3) return final text
    '''
    lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer("english")
    
    lemmatized_text = lemmatizer.lemmatize(text, pos='v')
    stemmed_text = stemmer.stem(lemmatized_text)
    
    return stemmed_text



# Tokenize and lemmatize. Look up how preprocessing is being done
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            preprocessed_token = lemmatize_stemming(token)
            result.append(preprocessed_token)
    return result
        

In [30]:
doc_sample = documents[documents['index'] == 2].values[0][1]
doc_sample

'a g calls for infrastructure protection summit'

In [31]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][1]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes

In [33]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents['headline_text'].map(preprocess)

In [34]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [46]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)


In [38]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [47]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
- keep 100000 most frequent tokens
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)



In [48]:
len(dictionary)

11988

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [49]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'.
Hint: Since we want a BoW model for each document, you can use list comprehension instead of for loops. List
comprehension is the more pythonic way.
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]


In [50]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(75, 1), (111, 1), (481, 1), (3948, 1)]

In [51]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 75 ("bushfir") appears 1 time.
Word 111 ("help") appears 1 time.
Word 481 ("rain") appears 1 time.
Word 3948 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [52]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [53]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
corpus_tfidf = tfidf[bow_corpus]



In [54]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5922233916923642),
 (1, 0.39421163662097713),
 (2, 0.4921004858653737),
 (3, 0.5017028519911626)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [55]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'

If Multicore is throwing errors on you laptop, use LDA mono-core, gensim.models.LdaModel
'''
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=2)


In [56]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.019*"charg" + 0.018*"polic" + 0.017*"murder" + 0.015*"court" + 0.013*"podcast" + 0.012*"woman" + 0.010*"assault" + 0.010*"search" + 0.010*"shoot" + 0.009*"miss"


Topic: 1 
Words: 0.015*"sport" + 0.014*"mine" + 0.010*"april" + 0.009*"stab" + 0.009*"coal" + 0.009*"drum" + 0.008*"monday" + 0.007*"outback" + 0.006*"jam" + 0.006*"reform"


Topic: 2 
Words: 0.011*"govern" + 0.010*"health" + 0.008*"fund" + 0.008*"asylum" + 0.008*"pacif" + 0.006*"seeker" + 0.006*"servic" + 0.006*"michael" + 0.006*"indonesia" + 0.005*"capit"


Topic: 3 
Words: 0.029*"rural" + 0.017*"news" + 0.015*"market" + 0.012*"busi" + 0.011*"farm" + 0.009*"rise" + 0.008*"nation" + 0.008*"share" + 0.007*"west" + 0.007*"price"


Topic: 4 
Words: 0.009*"abbott" + 0.008*"blaze" + 0.007*"premier" + 0.007*"novemb" + 0.007*"august" + 0.006*"decemb" + 0.006*"sell" + 0.006*"fiji" + 0.006*"intern" + 0.006*"senat"


Topic: 5 
Words: 0.010*"final" + 0.009*"quarter" + 0.009*"leagu" + 0.008*"plead" + 0.008*"grain" + 0

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: crime/murder/criminal case
* 1: mining/law reform
* 2: government/health/international
* 3: Rural / Business / Agriculture / Market
* 4: Politics / Abbott / Premier
* 5: Sports / Finals / League
* 6: News / Interview / Weather / Bushfires / Accidents
* 7: News / Country / Finance
* 8: News / Grandstand / Queensland / Murder
* 9: News / Election / Export / Politics / Dairy.

## Step 4.2 Running LDA using TF-IDF ##

In [58]:
'''
Define lda model using corpus_tfidf
'''
lda_model_tfidf = models.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=10)


In [59]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.021*"podcast" + 0.012*"share" + 0.011*"weather" + 0.009*"abus" + 0.009*"mine" + 0.008*"india" + 0.008*"japan" + 0.008*"child" + 0.008*"die" + 0.008*"melbourn"


Topic: 1 Word: 0.019*"march" + 0.015*"abbott" + 0.014*"live" + 0.012*"elect" + 0.011*"vote" + 0.009*"export" + 0.009*"station" + 0.009*"media" + 0.009*"drum" + 0.009*"toni"


Topic: 2 Word: 0.036*"hour" + 0.012*"search" + 0.011*"shoot" + 0.010*"island" + 0.009*"leagu" + 0.009*"woman" + 0.009*"miss" + 0.009*"cyclon" + 0.009*"polic" + 0.009*"hunter"


Topic: 3 Word: 0.013*"queensland" + 0.013*"hill" + 0.010*"brisban" + 0.009*"januari" + 0.009*"shire" + 0.008*"reveal" + 0.008*"port" + 0.007*"capit" + 0.006*"school" + 0.006*"student"


Topic: 4 Word: 0.017*"sydney" + 0.017*"murder" + 0.014*"grandstand" + 0.013*"speak" + 0.012*"australian" + 0.010*"charg" + 0.010*"breakfast" + 0.010*"sale" + 0.009*"court" + 0.009*"trial"


Topic: 5 Word: 0.040*"interview" + 0.011*"season" + 0.011*"market" + 0.010*"tasmania" + 0.009*

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: Podcasts, weather
* 1: Elections, media
* 2: Shootings, law enforcement
* 3: Queensland, schools
* 4: Murder trials, courts
* 5: Interviews, local markets
* 6: Rural issues, agriculture
* 7: Sports, geography
* 8: News, farming, law enforcement
* 9: Accidents, law enforcement, Royalty

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model

We will check to see where our test document would be classified. 

In [60]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [61]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8199657201766968	 
Topic: 0.042*"interview" + 0.017*"weather" + 0.010*"crash" + 0.010*"fatal" + 0.009*"bushfir" + 0.007*"driver" + 0.007*"victoria" + 0.007*"royal" + 0.007*"drive" + 0.007*"wednesday"

Score: 0.02001371793448925	 
Topic: 0.029*"countri" + 0.028*"hour" + 0.009*"wrap" + 0.008*"david" + 0.007*"januari" + 0.006*"drought" + 0.006*"western" + 0.006*"highlight" + 0.006*"shire" + 0.006*"invest"

Score: 0.020004399120807648	 
Topic: 0.029*"rural" + 0.017*"news" + 0.015*"market" + 0.012*"busi" + 0.011*"farm" + 0.009*"rise" + 0.008*"nation" + 0.008*"share" + 0.007*"west" + 0.007*"price"

Score: 0.020004330202937126	 
Topic: 0.011*"govern" + 0.010*"health" + 0.008*"fund" + 0.008*"asylum" + 0.008*"pacif" + 0.006*"seeker" + 0.006*"servic" + 0.006*"michael" + 0.006*"indonesia" + 0.005*"capit"

Score: 0.020003367215394974	 
Topic: 0.014*"grandstand" + 0.011*"queensland" + 0.010*"south" + 0.009*"kill" + 0.009*"februari" + 0.008*"john" + 0.007*"toni" + 0.007*"octob" + 0.006*"de

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model

In [62]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
Code would be similar to above
'''
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.682425320148468	 
Topic: 0.033*"countri" + 0.014*"april" + 0.013*"crash" + 0.009*"sentenc" + 0.009*"royal" + 0.009*"pacif" + 0.009*"driver" + 0.009*"ash" + 0.008*"polic" + 0.008*"fatal"

Score: 0.15753670036792755	 
Topic: 0.055*"rural" + 0.014*"drought" + 0.009*"sign" + 0.009*"farmer" + 0.009*"dairi" + 0.009*"tasmanian" + 0.008*"senat" + 0.008*"john" + 0.007*"food" + 0.007*"coal"

Score: 0.020010273903608322	 
Topic: 0.040*"interview" + 0.011*"season" + 0.011*"market" + 0.010*"tasmania" + 0.009*"local" + 0.009*"asylum" + 0.008*"job" + 0.007*"harvest" + 0.007*"david" + 0.007*"centr"

Score: 0.020007122308015823	 
Topic: 0.013*"queensland" + 0.013*"hill" + 0.010*"brisban" + 0.009*"januari" + 0.009*"shire" + 0.008*"reveal" + 0.008*"port" + 0.007*"capit" + 0.006*"school" + 0.006*"student"

Score: 0.02000468596816063	 
Topic: 0.036*"hour" + 0.012*"search" + 0.011*"shoot" + 0.010*"island" + 0.009*"leagu" + 0.009*"woman" + 0.009*"miss" + 0.009*"cyclon" + 0.009*"polic" + 0.009*"hunt

## Step 6: Testing model on unseen document ##

In [63]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.22000007331371307	 Topic: 0.015*"sport" + 0.014*"mine" + 0.010*"april" + 0.009*"stab" + 0.009*"coal"
Score: 0.22000005841255188	 Topic: 0.012*"march" + 0.012*"live" + 0.010*"dairi" + 0.008*"abus" + 0.007*"export"
Score: 0.2199999988079071	 Topic: 0.042*"interview" + 0.017*"weather" + 0.010*"crash" + 0.010*"fatal" + 0.009*"bushfir"
Score: 0.21999990940093994	 Topic: 0.029*"countri" + 0.028*"hour" + 0.009*"wrap" + 0.008*"david" + 0.007*"januari"
Score: 0.019999999552965164	 Topic: 0.019*"charg" + 0.018*"polic" + 0.017*"murder" + 0.015*"court" + 0.013*"podcast"
Score: 0.019999999552965164	 Topic: 0.011*"govern" + 0.010*"health" + 0.008*"fund" + 0.008*"asylum" + 0.008*"pacif"
Score: 0.019999999552965164	 Topic: 0.029*"rural" + 0.017*"news" + 0.015*"market" + 0.012*"busi" + 0.011*"farm"
Score: 0.019999999552965164	 Topic: 0.009*"abbott" + 0.008*"blaze" + 0.007*"premier" + 0.007*"novemb" + 0.007*"august"
Score: 0.019999999552965164	 Topic: 0.010*"final" + 0.009*"quarter" + 0.009*"le

### Now, look up different ways we can transform our words or use different techniques or improvements that we can make to make our Topic Classifier more accurate. Write them down below:

**We can:**
- Double click here to append points

- Fine tune the hyperparameters such as the number of topics, the number of topic words, the number of passes, etc.

- Preprocess more like removing stop words and punctuation.

- Using pre-trained word embeddings such as Word2Vec or GloVe

- Data augmentation